In [1]:
# import statements
import sys
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
# ===================================================
# initialize run environment as non-function calls
# so i don't pass 100000 variables in to a function
learning_rate = 0.01
momentum = 0.5
log_interval = 100
n_epochs = 3
mnist_global_mean = 0.1307
mnist_global_stdev = 0.3081
random_seed = 1
batch_size_train = 64
batch_size_test = 1000
epochs = 5
train_losses = []
train_counter = []
test_losses = []
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
# ===================================================

In [13]:
class NeuralNetwork(nn.Module):
    
    # channelSize is on the increment
    def __init__(self, convoCount, channelSize, dropoutRate, pixelSize):
        super(NeuralNetwork, self).__init__()

        # Feature Extraction Stage
        self.feature_maps_stack = nn.Sequential()
        startChannel = 1
        
        for count in range(convoCount):
            self.feature_maps_stack.append(nn.Conv2d(in_channels=startChannel,
                                                         out_channels=channelSize * (count + 1),
                                                         kernel_size=5))
            pixelSize -= 4
            print(pixelSize)
            
            if count >= 1:
                self.feature_maps_stack.append(nn.Dropout2d(p=dropoutRate))
                
            if count >= 1 and count < 3:
                self.feature_maps_stack.append(nn.MaxPool2d(kernel_size=2))
                # print("Before divide by 2")
                # print(pixelSize)
                pixelSize = int(pixelSize / 2)
                # print("After divide by 2")
                # print(pixelSize)
                
            self.feature_maps_stack.append(nn.ReLU())
            
            # reset env
            startChannel = channelSize * (count + 1)
        
        totalNode = pixelSize * pixelSize * startChannel
        
        # Classification Stage
        self.classify_stack = nn.Sequential(
            # A flattening operation 
            nn.Flatten(),
            # followed by a fully connected Linear layer with 50 nodes 
            nn.Linear(totalNode, 50),
            # and a ReLU function on the output
            nn.ReLU(),
            # A final fully connected Linear layer with 10 nodes
            nn.Linear(50, 10)
        )
        
    def forward(self, x):
        features = self.feature_maps_stack(x)
        logits = self.classify_stack(features)
        
        # and the log_softmax function applied to the output
        return nn.functional.log_softmax(logits, dim=1)

In [20]:
model = NeuralNetwork(3, 30, 0.5, 28)

# print(model)
test = str(model)
print(test)

24
20
6
NeuralNetwork(
  (feature_maps_stack): Sequential(
    (0): Conv2d(1, 30, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(30, 60, kernel_size=(5, 5), stride=(1, 1))
    (3): Dropout2d(p=0.5, inplace=False)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU()
    (6): Conv2d(60, 90, kernel_size=(5, 5), stride=(1, 1))
    (7): Dropout2d(p=0.5, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): ReLU()
  )
  (classify_stack): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=810, out_features=50, bias=True)
    (2): ReLU()
    (3): Linear(in_features=50, out_features=10, bias=True)
  )
)
